In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


In [2]:
file = r.TFile("events2W.root","READ")
tree = file.Get("tree") # Getting the tree 

Warning in <TFile::Init>: file events2W.root probably not closed, trying to recover
Info in <TFile::Recover>: events2W.root, recovered key TTree:tree at address 2187
Warning in <TFile::Init>: successfully recovered 1 keys


In [3]:
# Initializing 4-element array to store 4-vectors
w1Arr = np.zeros(4)
w2Arr = np.zeros(4)
jet1Arr = np.zeros(4)
jet2Arr = np.zeros(4)
eArr = np.zeros(4)
nArr = np.zeros(4)

In [4]:
# Taking already filled branches from the file
# Now we need to add another branch for neutrino
tree.SetBranchAddress("w1Boson", w1Arr)
tree.SetBranchAddress("w2Boson", w2Arr)
tree.SetBranchAddress("jet1", jet1Arr)
tree.SetBranchAddress("jet2", jet2Arr)
tree.SetBranchAddress("electron", eArr)

# Initializing branch for neutrino components
tree.Branch("neutrino",nArr,"neArray[4]/D")

In [5]:
#tree.Scan()

In [6]:
%%cpp -d   

void fcn(int &npar, double *gin, double &f, double *par, int iflag){
    double pTSq = par[0];
    double pTotSq = par[1];
    double ePz = par[2];
    double nPz = par[3];

    double wMass = TMath::Sqrt(2*pTSq + 2 * TMath::Sqrt(pTotSq) * TMath::Sqrt(pTSq + pow(nPz,2)) - 2 * ePz * nPz);

    f = pow(wMass-80.3,2);
};

double minimization_func(double *wArr, double *elArr){
    double ePx = elArr[0];
    double ePy = elArr[1];
    double ePz = elArr[2];
    
    double wPz = wArr[2]; 
    
    double initVal;
    
    if(abs(ePz - wPz) > 1){
        initVal = -(ePz - wPz)+1;
    } else {
        initVal = -ePz + 0.02;  
    }
    
    double pTSqEl = pow(ePx,2) + pow(ePy,2);
    double pTotSqEl = pTSqEl + pow(ePz,2);  
    
    TMinuit *gMinuit = new TMinuit(4);
    gMinuit->SetFCN(fcn);
    gMinuit->DefineParameter(0, "pTotSqEl", pTSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(1, "pTSqEl", pTotSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(2, "ePz", ePz, 1., 0., 0.);
    gMinuit->DefineParameter(3, "nPz", initVal, 0.05, 0., 0.);
    gMinuit->FixParameter(0);
    gMinuit->FixParameter(1);
    gMinuit->FixParameter(2);
    gMinuit->Command("MIGRAD");

    double nPz, nPzErr;
    gMinuit->GetParameter(3, nPz, nPzErr);
    return nPz;

};

In [7]:
n = tree.GetEntries() # Number of entries in tree (number of different events)

for i in range(n):
    tree.GetEntry(i)
    ePx = tree.GetListOfLeaves().At(4).GetValue(0)
    ePy = tree.GetListOfLeaves().At(4).GetValue(1)
    
    nArr[0] = -ePx
    nArr[1] = -ePy
    nArr[2] = r.minimization_func(w2Arr, eArr)
    nArr[3] = r.TMath.Sqrt(nArr[0]**2 + nArr[1]**2 + nArr[2]**2)
    
    tree.GetBranch("neutrino").Fill() # Filling just "neutrino" branch

 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 pTotSqEl     8.10367e+02  1.00000e+00     no limits
     2 pTSqEl       1.97797e+03  1.00000e+00     no limits
     3 ePz          3.41703e+01  1.00000e+00     no limits
     4 nPz         -2.20859e+01  5.00000e-02     no limits
 **********
 **    1 **FIX           1
 **********
 **********
 **    2 **FIX           2
 **********
 **********
 **    3 **FIX           3
 **********
 **********
 **    4 **MIGRAD 
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-04
 FCN=0.501069 FROM MIGRAD    STATUS=INITIATE        4 CALLS           5 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS       STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  pTotSqEl     8.10367e+02     fixed    
 

In [8]:
#Scanning just for W (decaying into electron and neutrino), electron and neutrino
tree.Scan("w2Boson:electron:neutrino")

40

***********************************************************
*    Row   * Instance *   w2Boson *  electron *  neutrino *
***********************************************************
*        0 *        0 * -0.064570 * -27.69305 * 27.693055 *
*        0 *        1 * -0.053423 * -6.592521 * 6.5925212 *
*        0 *        2 * 11.084390 * 34.170258 * -23.00186 *
*        0 *        3 * 81.005930 * 44.474411 * 36.598528 *
*        1 *        0 * 0.5674849 * 7.9521592 * -7.952159 *
*        1 *        1 * 0.6005204 * 15.679958 * -15.67995 *
*        1 *        2 * 10.968961 * -31.11260 * 41.705226 *
*        1 *        3 * 80.994387 * 35.736427 * 45.259516 *
*        2 *        0 * 0.8218536 * 39.940439 * -39.94043 *
*        2 *        1 * 1.5629852 * 5.5645789 * -5.564578 *
*        2 *        2 * 10.507879 * 0.0980422 * 0.0981818 *
*        2 *        3 * 80.948279 * 40.326329 * 40.326329 *
*        3 *        0 * -0.005370 * 1.7267849 * -1.726784 *
*        3 *        1 * -1.015632 * -0.3

Type <CR> to continue or q to quit ==> 